In [1]:
import os
while os.getcwd().split('\\')[-1] != 'ecological-networks':
    %cd ..
import setup_paths
setup_paths.add_path()
from Species_Reintroductions import *
DATA_TYPE = 'Real_1D'
CASE_TO_SOLVE_init(DATA_TYPE)
get_CASE()

d:\Github\ecological-networks\Code\01_Species_Reintroductions
d:\Github\ecological-networks\Code
d:\Github\ecological-networks
Real_1D


In [ ]:
def G_iter(data_two):
    for key in list(data_two.keys()):
        for ensembles_ID in range(len(data_two[key])):
            yield (key, ensembles_ID, 0)

def get_ImpData(NetworkName, loc):
    M_df = M[NetworkName]
    data_two = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"{NetworkName}", f"{NetworkName}-Data_two.pkl"))
    Keys_Filter = [[0,1,2], [2,5,8], [0]]
    Ensembles_batch = [0,10]
    data_two = {k: v[Ensembles_batch[0]:min(Ensembles_batch[1],len(v))] for k,v in data_two.items() if k in set(list(itertools.product(*Keys_Filter)))}
    del Keys_Filter, Ensembles_batch
    Data_Imp = []
    for approach in ['three','four','five','six','rand']:
        for key, E, T in itertools.islice(G_iter(data_two),0,None,1):
            try:
                X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", NetworkName, f"{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl"))
                #print(FileName)
                M_per = RemovalFromNodeList(data_two[key][E],M_df,axis=key[2])
                M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                
                M_per_prev = {}
                M_per_next = {}
                
                Data_Temp = []
                
                for Step in list(range(1,len(X[0])+1)):
                    M_per_prev['S'] = M_per.shape[0] + M_per.shape[1]
                    M_per_prev['A'] = M_per.shape[1] / M_per.shape[0]
                    M_per_prev['C'] = sum(np.array(M_per).flatten()) / (M_per.shape[1]*M_per.shape[0])
                    M_per_prev['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                    M_per_prev['X'] = np.mean(np.array(X[3][Step-1][-1,:]))
                    M_per_prev['X_ob'] = np.mean(np.array(list(map(isExtinct,X[3][Step-1][-1,:]))))

                    Node_Choice = X[0][Step-1]
                    M_per = M_per.append(M_df.loc[X[0][Step-1]])
                    M_per[M_per.isnull()] = M_df
                    M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                    M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                    NetM, _, _ = getNetDefinitions(M_per,None,None, flag = [True, False, False])
                    
                    Node_D = nx.degree_centrality(NetM)[Node_Choice]
                    Node_C = nx.closeness_centrality(NetM)[Node_Choice]
                    Node_B = nx.betweenness_centrality(NetM)[Node_Choice]
                    
                    M_per_next['S'] = M_per.shape[0] + M_per.shape[1]
                    M_per_next['A'] = M_per.shape[1] / M_per.shape[0]
                    M_per_next['C'] = sum(np.array(M_per).flatten()) / (M_per.shape[1]*M_per.shape[0])
                    M_per_next['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                    M_per_next['X'] = np.mean(np.array(X[3][Step][-1,:]))
                    M_per_next['X_ob'] = np.mean(np.array(list(map(isExtinct,X[3][Step][-1,:]))))

                    #temp = [M_per_next[x] - M_per_prev[x] for x in ['S','A','C','N','X','X_ob','ST','P']] + [M_per_next[x] / M_per_prev[x] for x in ['S','A','C','N','X','X_ob','ST','P']]
                    temp = [M_per_next[x] - M_per_prev[x] for x in ['S','A','C','N','X','X_ob']] + [M_per_next[x] / M_per_prev[x] for x in ['S','A','C','N','X','X_ob']]
                    Data_Temp += [temp]
                Data_Temp = np.array(Data_Temp)
                Data_Temp = np.mean(Data_Temp, axis = 0)
                
                Data_Imp += [[approach, key[0], key[1], key[2], E] + list(Data_Temp)]
            except:
                print(f"ERROR: {NetworkName}, {key}, {E}, {T}")
                #pass

    DF = pd.DataFrame(Data_Imp, columns = ['Approach','Key0','Key1','Key2','E','S0','A0','C0','N0','X0','X_ob0','S1','A1','C1','N1','X1','X_ob1'])
    #DF.drop_duplicates(inplace=True, ignore_index=True)
    '''
    lst = ['S','A','C','N','X','X_ob','ST','P']
    for i,j,k,l in zip(list(DF.columns)[3:11], list(DF.columns)[11:], [x + '_diff' for x in lst], [x + '_r' for x in lst]):
        DF[k] = DF[j] - DF[i]
        DF[l] = DF[j]/DF[i]
    '''
    #print(NetworkName, len(Data_Scatter), DF.shape[0])
    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '04_Importance_Data', 'ImportanceData', f"{loc}", f"{NetworkName}.pkl")
    os.makedirs(os.path.dirname(FileName), exist_ok=True)
    PickleObj(DF,FileName)

In [ ]:
for CASE_TO_SOLVE, loc in zip(['Real_1D', 'Syn_01', 'Syn_2D', 'Real_2D'],['01','02','03','04']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))

    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}

    #with Parallel(n_jobs=-1, verbose = 10) as parallel:
            #data = parallel(delayed(get_ImpData)(NetworkName,loc) for NetworkName in itertools.islice(M.keys(),0,None,1))

In [21]:
DATA_TYPE = 'Real_2D'
CASE_TO_SOLVE_init(DATA_TYPE)
M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
loc = '04'
if DATA_TYPE.split('_')[1] == '1D':
    approach_lst = ['three','four','five','six','rand']
elif DATA_TYPE.split('_')[1] == '2D':
    approach_lst = ['four','five','six','rand']

Data = {}
for NetworkName in itertools.islice(M.keys(),0,None,1):
    DF = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '04_Importance_Data', 'ImportanceData', f"{loc}", f"{NetworkName}.pkl"))
    DF.drop(['N1','X_ob1'], axis = 'columns', inplace = True)
    DF = DF.dropna()
    for col in list(DF.columns):
        DF.drop(list(DF[ DF[col] == np.inf ].index), inplace = True)
    Data[NetworkName] = DF
del M, loc, DF, NetworkName, col
#
for NetworkName in itertools.islice(list(Data.keys()),0,None,1):
    #DF = pd.DataFrame([], columns = ['Approach','Key0','Key1','Key2','S0','A0','C0','N0','X0','X_ob0','S1','A1','C1','N1','X1','X_ob1'])
    DF = pd.DataFrame([], columns = ['Approach','Key0','Key1','Key2','S0','A0','C0','N0','X0','X_ob0','S1','A1','C1','X1'])
    for approach in approach_lst:
        for key in set(list(itertools.product(*[[0,1,2], [2,5,8], [0]]))):
            try:
                DF_Slice = Data[NetworkName]
                
                DF_Slice = DF_Slice[ (DF_Slice['Approach'] == approach) & (DF_Slice['Key0'] == key[0]) & (DF_Slice['Key1'] == key[1]) & (DF_Slice['Key2'] == key[2])]
                X = np.array(DF_Slice)[:,5:]
                Y = list(np.mean(X, axis = 0))
                Y = [approach, key[0], key[1], key[2]] + Y
                DF = DF.append(pd.Series(Y, index = DF.columns), ignore_index=True)
            except:
                print(NetworkName, approach, key)
                Y = [approach, key[0], key[1], key[2]] + [np.nan for i in range(len(DF.columns)-4)]
                DF = DF.append(pd.Series(Y, index = DF.columns), ignore_index=True)
    DF.sort_values(by = ['Key0','Key1','Key2'], ascending = True, inplace = True)
    Data[NetworkName] = DF
del NetworkName, approach, key, DF_Slice, X, Y, DF, approach_lst

In [ ]:
print(type(Data))
print(len(Data.keys()))
print(Data.keys())
print(type(Data['M_PL_041']))
print(Data['M_PL_041'].shape)
print(Data['M_PL_041'].columns)
Data['M_PL_041'].head()

In [23]:
Data[np.random.choice(list(Data.keys()))].columns

Index(['Approach', 'Key0', 'Key1', 'Key2', 'S0', 'A0', 'C0', 'N0', 'X0',
       'X_ob0', 'S1', 'A1', 'C1', 'X1'],
      dtype='object')

In [19]:
if DATA_TYPE.split('_')[1] == '1D':
    approach_lst = ['three','four','five','six','rand']
elif DATA_TYPE.split('_')[1] == '2D':
    approach_lst = ['four','five','six','rand']
Z = []
for approach in approach_lst:
    D = []
    for NetworkName in itertools.islice(list(Data.keys()),0,None,1):
        DF = Data[NetworkName]
        #DF = DF.loc[:,['Approach','Key0','Key1','Key2','X1']]
        row = []
        for key in zip([0,0,0,1,1,1,2,2,2],[2,5,8]*3, [0]*9):
            val = DF[ (DF['Approach'] == approach) & (DF['Key0'] == key[0]) & (DF['Key1'] == key[1]) & (DF['Key2'] == key[2])]['X1'].values[0]
            row += [val]
        D += [row]
    D = np.array(D)
    del DF, NetworkName,row, val,key,approach
    Z += [np.mean(D, axis = 0)]
Z = np.array(Z)
Z = pd.DataFrame(Z, index = approach_lst)
round((Z - 1)*100,3)

,0,1,2,3,4,5,6,7,8
three,57.505,72.086,154.897,183.505,185.984,159.274,46.179,32.539,59.977
four,59.855,73.008,160.158,183.963,187.405,167.697,47.572,31.596,59.869
five,61.512,73.488,159.575,182.798,187.828,168.356,47.407,31.425,60.828
six,61.778,73.191,159.213,184.193,189.882,169.828,47.887,32.108,59.697
rand,61.249,73.883,142.531,186.411,188.911,156.395,48.623,31.919,56.910


In [22]:
if DATA_TYPE.split('_')[1] == '1D':
    approach_lst = ['three','four','five','six','rand']
elif DATA_TYPE.split('_')[1] == '2D':
    approach_lst = ['four','five','six','rand']
Z = []
for approach in approach_lst:
    D = []
    for NetworkName in itertools.islice(list(Data.keys()),0,None,1):
        DF = Data[NetworkName]
        #DF = DF.loc[:,['Approach','Key0','Key1','Key2','X1']]
        row = []
        for key in zip([0,0,0,1,1,1,2,2,2],[2,5,8]*3, [0]*9):
            val = DF[ (DF['Approach'] == approach) & (DF['Key0'] == key[0]) & (DF['Key1'] == key[1]) & (DF['Key2'] == key[2])]['X1'].values[0]
            row += [val]
        D += [row]
    D = np.array(D)
    del DF, NetworkName,row, val,key,approach
    Z += [np.mean(D, axis = 0)]
Z = np.array(Z)
Z = pd.DataFrame(Z, index = approach_lst)
round((Z - 1)*100,3)

,0,1,2,3,4,5,6,7,8
four,0.608,1.183,2.735,2.227,2.774,4.006,0.283,0.511,1.218
five,0.608,1.183,2.734,2.226,2.772,4.005,0.283,0.511,1.217
six,0.608,1.184,2.736,2.228,2.775,4.006,0.283,0.511,1.219
rand,0.605,1.162,2.558,2.204,2.671,3.669,0.282,0.507,1.197


In [36]:
DF_Full = pd.DataFrame([], columns = Data[np.random.choice(list(Data.keys()))].columns)
approach = 'six'
for NetworkName in itertools.islice(list(Data.keys()),0,None,1):
    DF_Slice = Data[NetworkName].copy()
    DF_Slice = DF_Slice[DF_Slice['Approach'] == approach]
    DF_Full = DF_Full.append(DF_Slice, ignore_index=True)
DF_Full.corr()['X1']

S0       0.740117
A0       0.269969
C0      -0.508399
N0       0.652208
X0       0.940814
X_ob0    0.940814
S1       0.900293
A1       0.727000
C1      -0.480131
X1       1.000000
Name: X1, dtype: float64

In [29]:
DF_Full = pd.DataFrame([], columns = Data[np.random.choice(list(Data.keys()))].columns)
approach = 'six'
for NetworkName in itertools.islice(list(Data.keys()),0,None,1):
    DF_Slice = Data[NetworkName].copy()
    DF_Slice = DF_Slice[DF_Slice['Approach'] == approach]
    DF_Full = DF_Full.append(DF_Slice, ignore_index=True)
DF_Full.corr()

,S0,A0,C0,N0,X0,X_ob0,S1,A1,C1,X1
S0,1.000000,0.268595,-0.180740,0.439302,0.670032,0.670032,0.576740,0.535378,-0.211328,0.740117
A0,0.268595,1.000000,0.484235,0.480201,0.229626,0.229626,0.317721,0.595014,0.426539,0.269969
C0,-0.180740,0.484235,1.000000,0.064428,-0.429249,-0.429249,-0.547942,-0.127641,0.934867,-0.508399
N0,0.439302,0.480201,0.064428,1.000000,0.635967,0.635967,0.570733,0.671060,0.098616,0.652208
X0,0.670032,0.229626,-0.429249,0.635967,1.000000,1.000000,0.744199,0.567559,-0.390512,0.940814
X_ob0,0.670032,0.229626,-0.429249,0.635967,1.000000,1.000000,0.744199,0.567559,-0.390512,0.940814
S1,0.576740,0.317721,-0.547942,0.570733,0.744199,0.744199,1.000000,0.866406,-0.531858,0.900293
A1,0.535378,0.595014,-0.127641,0.671060,0.567559,0.567559,0.866406,1.000000,-0.134499,0.727000
C1,-0.211328,0.426539,0.934867,0.098616,-0.390512,-0.390512,-0.531858,-0.134499,1.000000,-0.480131
X1,0.740117,0.269969,-0.508399,0.652208,0.940814,0.940814,0.900293,0.727000,-0.480131,1.000000
